# HBV demo (Colab-ready)

This notebook runs a minimal HBV implementation (`hbv.py`) using example inputs in `examples/`.

- Inputs: daily forcing (Time, Precipitation, Temperature) and monthly PET climatology (month, T_avg_month, PEm_day)
- Outputs: all states and fluxes returned by `hbv_run`, plus NSE against observed discharge (if provided)

**Retina figures** are enabled for high-resolution plots.


In [ ]:
# Colab setup (optional)
# If running from GitHub via Colab, uncomment and edit:
# !git clone https://github.com/<YOUR_GITHUB_USERNAME>/<YOUR_REPO_NAME>.git
# %cd <YOUR_REPO_NAME>

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# High-resolution figures (retina)
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.dpi"] = 150


In [ ]:
# Imports
from hbv import hbv_run, nse


## Parameters
Default parameter set (ParamInitial).

In [ ]:
ParamInitial = dict(
    dd=6.10,
    fc=195.0,
    beta=2.6143,
    c=0.07,
    k0=0.163,
    l0=4.87,
    k1=0.029,
    k2=0.049,
    kp=0.050,
    pwp=106.0,
)

params = np.array([
    ParamInitial["dd"],
    ParamInitial["fc"],
    ParamInitial["beta"],
    ParamInitial["c"],
    ParamInitial["k0"],
    ParamInitial["l0"],
    ParamInitial["k1"],
    ParamInitial["k2"],
    ParamInitial["kp"],
    ParamInitial["pwp"],
], dtype=float)

params


## Load example inputs
This expects the example folder shipped with the repo.

In [ ]:
from pathlib import Path

# Example data folder (adjust if needed)
example_dir = Path("examples") / "Prof_Amir_AghaKouchak_example"
print("Example directory:", example_dir.resolve())

forcing_path = example_dir / "forcing.csv"
pet_path = example_dir / "pet_monthly.csv"
qobs_path = example_dir / "Qobs.csv"

forcing = pd.read_csv(forcing_path, parse_dates=["Time"])
pet = pd.read_csv(pet_path)

forcing.head(), pet.head()


## Run HBV

In [ ]:
results, aux = hbv_run(
    forcing=forcing,
    pet_monthly=pet,
    params=params,
    area_km2=410.0,
    Tsnow_thresh=0.0,
    init_state={"snow": 0.0, "soil": 0.0, "s1": 0.0, "s2": 0.0},
)

results.head()


## Evaluate against observed discharge (optional)

In [ ]:
if qobs_path.exists():
    qobs = pd.read_csv(qobs_path, parse_dates=["Time"])
    # Align on Time (inner join)
    merged = results.merge(qobs, on="Time", how="inner", suffixes=("", "_obs"))
    # Expect observed column name Q_m3s or Qobs; adjust if needed
    obs_col = "Q_m3s_obs" if "Q_m3s_obs" in merged.columns else ("Qobs" if "Qobs" in merged.columns else None)
    if obs_col is None:
        print("Observed discharge column not found. Available columns:", list(qobs.columns))
    else:
        score = nse(merged[obs_col].to_numpy(), merged["Q_m3s"].to_numpy())
        print("NSE:", score)
else:
    print("No observed discharge file found at:", qobs_path)


## Visualize all states and fluxes

In [ ]:
vars_to_plot = [
    "snow", "liq_water", "pe", "ea", "soil", "dq", "s1", "s2", "q_mmday", "Q_m3s"
]

t = results["Time"]

for v in vars_to_plot:
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(t, results[v])
    ax.set_title(v)
    ax.set_xlabel("Time")
    ax.set_ylabel(v)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## Quick discharge plot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(results["Time"], results["Q_m3s"], label="Simulated")
if qobs_path.exists() and 'merged' in locals() and obs_col is not None:
    ax.plot(merged["Time"], merged[obs_col], label="Observed", alpha=0.8)
ax.set_xlabel("Time")
ax.set_ylabel("Discharge (m$^3$ s$^{-1}$)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
